In [1]:
import pandas as pd
import numpy as np
import sklearn
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer
import pyreadr
import os

## Learning goals

Be able to...

- define and utilize different data splitting techniques,
- compare and interpret evaluation results depending on the experimental designs.

## Introduction

From previous exercise:

In [ ]:
# Load data:
data_ctg = pyreadr.read_r(os.path.join(os.getcwd(), "data_ctg.rds"))[None]

# split into X and y:
y = data_ctg.pop("status")
X = data_ctg

#change "Year" to integer (otherwise issues arise with Logistic Regression):
X["Year"] = X["Year"].astype(int)
# change  y to a 0/1 encoding; the models would probably work with the current encoding, but this is more controlleable:
y = y.replace({"normal": "0", "suspect": "1"})
y = y.astype(int)

# if you think further data preprocessing is necessary, feel free to add it here


## Setup: algorithms \& hyperparamers 

This section shows a very basic implemkentation of hyperparameter searches. You will likely have to re-use so of this code later for your custom cross-validation strategies. 

Unlike in mlr3, there is (to my knowledge) no package that defines recommended hyperparameter search spaces for you in python.
You usually have to come up with sensible values yourself and define a space and search algorithm. As we aren't **that** concerned with cutting-edge performance today, I used filled  in some values that seemed plauisible to me (Tom).

For a pointer what these "sensible" values might usually be, you can of course always reference the values that mlr3 uses (the "vals" give an upper and lower bound for each hyperparameter):
<https://github.com/mlr-org/mlr3tuningspaces/blob/main/R/tuning_spaces_default.R>
Keep in mind that the hyperparameters might be named differently or that there might be subtle differences in implementations. For example, sklearn's LogisticRegression expects a parameter C which is the **inverse** of the regularization strength.

You could then, for each hyperparameter, define multiple values in between those bounds, e.g. np.linspace() or range() and use a grid search to find the best value from the grid:
<https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV>
Note: it is probably a good idea to run multiple runs in parallel using the `n_jobs` parameter.

The GridSearchCV object can be initialized with a custom `cv` parameter. This determines the cross-validation strategy. The default is `cv=5`, meaning the algorithm will do a 5-fold split to determine the best model on average.
We will later ajust this for custom strategies and just leave it as is for now (or set it to `cv=2` to reduce the computational load).

You could (maybe not today) also use more sophisticated search algorithms that maybe find a better/faster solution:
<https://scikit-learn.org/stable/modules/grid_search.html>

You might also want to give the metric you're using to measure model performance some thought. We will use the ROC-AUC which is a standard metric for binary classifiers.


In [15]:
# define the metric we will use to compare model performance:
scorer = make_scorer(roc_auc_score, response_method="predict_proba")

In [29]:
# How many hyperparameters should we sample randomly per algorithm? Notice that for a grid search, this is not as straightforward as in mlr3 since we don't have a function that magically fills the search space with a given number of samples.
# We have to manually define how many values we want to try for each hyperparameter.
# for simplicity, we will just use two different values depending on how many hyperparameters we have to tune; usually we would probably prefer these to be larger:
n_hyp_small = 10
n_hyp_large = 50

# Define the model for which we search hyperparameters; GridSearch expects an initialized model object and will then set the hyperparameters on a model copy internally.
elastic_net_classifier = LogisticRegression(penalty='elasticnet', solver = "saga", l1_ratio=0.5, max_iter=10000)

# define a grid of hyperparameters to search over:
glmnet_hyperparam_grid = {"C": np.linspace(0.01, 100, n_hyp_large)}

# define the gridsearch object:
ex_glmnet_hyperparam_gridsearch = GridSearchCV(elastic_net_classifier, glmnet_hyperparam_grid, scoring=scorer, cv=2, n_jobs=-1, verbose = 1)

In [26]:
# optional: run the gridsearch; we will later use custom cross-validation strategies, so this data will not be used again:
ex_glmnet_hyperparam_gridsearch.fit(X, y)
print("Best hyperparameters for glmnet:", ex_glmnet_hyperparam_gridsearch.best_params_)
# Note that one could also have used LogisticRegressionCV with an inbuilt cross-validation

Fitting 2 folds for each of 50 candidates, totalling 100 fits
Best hyperparameters for glmnet: {'C': 57.14714285714285}


In [ ]:
# Same thing for a random forest classifier:
rf_classifier = RandomForestClassifier(bootstrap=True)

rf_hyperparam_grid = {
    "n_estimators": np.linspace(1, 2000, n_hyp_small, dtype=int),           # corresponds to num.trees in mlr3
    "max_features": np.linspace(0.1, 1.0, n_hyp_small//2),       # corresponds to mtry.ratio (fraction of features consiodered at each split) in mlr3
    "max_samples": np.linspace(0.1, 1.0, n_hyp_small//2)         # corresponds to sample.fraction (fraction of samples used for each tree) in mlr3
}
ex_rf_hyperparam_gridsearch = GridSearchCV(rf_classifier, rf_hyperparam_grid, scoring=scorer, cv=2, n_jobs=-1, verbose = 1)


In [37]:
# optional: run the gridsearch; we will later use custom cross-validation strategies, so this data will not be used again:
ex_rf_hyperparam_gridsearch.fit(X, y)
print("Best hyperparameters for random forest:", ex_rf_hyperparam_gridsearch.best_params_)

Fitting 2 folds for each of 250 candidates, totalling 500 fits
Best hyperparameters for random forest: {'max_features': 0.1, 'max_samples': 0.55, 'n_estimators': 889}


## 2.1 Train-tune-test split (20 min)

### Task

Derive a random train-tune-test (60-20-20) split by explicitly setting three sets of indices (row_ids). Hereby, make sure to set a random seed (we'll use 42) and to stratify the splits for the outcome variable.

- `train_indices = ...`
- `tune_indices = ...`
- `test_indices = ...`

Optional: do a GridSearch on the obtained split

Even more Optional: test the best model on the test data set; how would you go about refitting the model on the combined train/tune data? There is a simple method; maybe have a look at the `GridSearchCV` documentation again.

### Very important hint: If you want to use your train/validation splits in the Grid-search, the easiest method I found is to pass a list of length 1 of tuples [(train_indices, tune_indices)]

Hint: 
- sklearn includes a function `train_test_split` which I would advise to use. However, there is no immediate 3-way split function
- `train_test_split` does not output indices; but if you input an array of indices of the right shape, you can get the right effect
- seeds in sklearn are set in each function; so for example train_test_split(..., random_state = 42)
- depending on the dataset, shuffling the data might be important
- check that your split worked as intended afterwards ( for ex. you could check the sizes of each dataset or the fractions of positive samples)

In [ ]:
# You can use this codebox for your solution or add additional ones if needed
# get train indices:
train_tune_indices, test_indices = train_test_split(np.arange(X.shape[0]), test_size=0.2, random_state=42, shuffle=True, stratify=y)
train_indices, tune_indices = train_test_split(train_tune_indices, test_size=0.25, random_state=42, shuffle=True, stratify=y.iloc[train_tune_indices])

In [ ]:
glmnet_hyperparam_gridsearch = GridSearchCV(elastic_net_classifier, glmnet_hyperparam_grid, scoring=scorer, cv = [(train_indices, tune_indices)], n_jobs=-1, verbose = 1, refit=True)
glmnet_hyperparam_gridsearch.fit(X, y)
print("Best hyperparameters for glmnet:", glmnet_hyperparam_gridsearch.best_params_)
X_test, y_test = X.iloc[test_indices], y.iloc[test_indices]
print(f"score on test data: {scorer(glmnet_hyperparam_gridsearch, X_test, y_test)}")

Fitting 1 folds for each of 50 candidates, totalling 50 fits
Best hyperparameters for glmnet: {'C': 2.050612244897959}
best score on tune data: 0.9510257544313273
score on test data: 0.962099819684959


## 2.2 Nested cross-validation

### Task

Define a nested CV design with 5 folds for the outer loop and a simple holdout design for the inner loop (75% training, 25% tuning).

I think there is two main ways of approaching this. 

The first, which I used in my solution, is to make use of the `cross_validate` function. For reference, see 
<https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html>
Essentially, you put the GridSearchCV as an estimator into the cross_validate.
The difficult part is, that there seems to be no easy way of telling the inner Gridsearch to just use a single split; a true nested cv would be much easier to implement :D

In my solution, I therefore implement a small `train_test_splitter()` class that implements a `split(self, X, y=None, groups=None)` function returning `[(train_indices, test_indices)]` and a `get_n_splits(self, *args, **kwargs)` function always returning 1. See also <https://scikit-learn.org/stable/glossary.html#term-CV-splitter>

As an alternative method, you could use the function KFold and essentially implement the loop manually.


In [60]:
class train_test_splitter():
    def __init__(self, train_size=0.75, random_state=42, shuffle=True):
        self.train_size = train_size
        self.random_state = random_state
        self.shuffle = shuffle

    def split(self, X, y=None, groups=None):
        n_samples = X.shape[0]
        print(n_samples)
        indices = np.arange(n_samples)
        rng = np.random.default_rng(self.random_state)
        if self.shuffle:
            rng.shuffle(indices)
        train_size = int(self.train_size * n_samples)
        print(train_size)
        train_indices = indices[:train_size]
        test_indices = indices[train_size:]
        return [(train_indices, test_indices)]

    def get_n_splits(self, *args, **kwargs):
        return 1
    
custom_cv = train_test_splitter(train_size=0.75, random_state=42)

glmnet_hyperparam_gridsearch = GridSearchCV(elastic_net_classifier, glmnet_hyperparam_grid, scoring=scorer, cv = custom_cv, n_jobs=-1, verbose = 1)
ncv_results = cross_validate(glmnet_hyperparam_gridsearch, X, y, cv=5, scoring=scorer, return_train_score=True, return_estimator=True, return_indices=True)

Fitting 1 folds for each of 50 candidates, totalling 50 fits
1656
1242
Fitting 1 folds for each of 50 candidates, totalling 50 fits
1656
1242
Fitting 1 folds for each of 50 candidates, totalling 50 fits
1656
1242
Fitting 1 folds for each of 50 candidates, totalling 50 fits
1656
1242
Fitting 1 folds for each of 50 candidates, totalling 50 fits
1656
1242


In [ ]:
# we join the results of all outer folds together:
all_results = pd.concat([pd.DataFrame(ncv_results["estimator"][i].cv_results_).assign(fold=i+1) for i in range(len(ncv_results["estimator"]))], ignore_index=True)
print(all_results.head())

   mean_fit_time  std_fit_time  mean_score_time  std_score_time   param_C  \
0       1.781557           0.0         0.002167             0.0  0.010000   
1       2.250076           0.0         0.002274             0.0  2.050612   
2       2.290004           0.0         0.002225             0.0  4.091224   
3       2.265292           0.0         0.003011             0.0  6.131837   
4       2.309574           0.0         0.002222             0.0  8.172449   

                     params  split0_test_score  mean_test_score  \
0               {'C': 0.01}           0.922340         0.922340   
1  {'C': 2.050612244897959}           0.932291         0.932291   
2  {'C': 4.091224489795918}           0.932291         0.932291   
3  {'C': 6.131836734693877}           0.932291         0.932291   
4  {'C': 8.172448979591836}           0.932291         0.932291   

   std_test_score  rank_test_score  fold  
0             0.0               50     1  
1             0.0               36     1  
2    

## 2.3 Temporal data split

### Task

Similar to tasks 2.1 and 2.2, create a temporal data split. For this, we want to create several splits where each time we use data from 
one year for training and data from the next year for testing.

- What is different for this splitting strategy compared to cross-validation?
- Are there other temporal splits that may be reasonable here? 

Hints:
- for implementation, I would suggest a manual approach similar to Ex. 2.1

In [ ]:
# You can use this codebox for your solution or add additional ones if needed

### Solution: 
- Internal-external validation (accessing temporal transferability) instead of internal validation.
In addition, not all subsets of the data have equal size.
- E.g. train in 2 years, test in the next year.

## 2.4 Model training and selection (for completeness only, no tasks here)

After hyperparameter tuning, we need to select models for testing/evaluation. 

We implemented some of the model tuning and selection in the above exercises, but to ensure that there is a consistent starting point for this exercise, we will rely on data obtained from the `"scripts/2_rwd_ctg_model_dev"` script.


In [64]:
data_eval_ttt_2 = pyreadr.read_r(os.path.join(os.getcwd(), "data_eval_ttt_2.rds"))[None]
print(data_eval_ttt_2.head())

   row_ids   truth response_glmnet  prob.suspect_glmnet  prob.normal_glmnet  \
0        3  normal          normal             0.026135            0.973865   
1       19  normal          normal             0.006974            0.993026   
2       28  normal          normal             0.000184            0.999816   
3       31  normal          normal             0.000017            0.999983   
4       47  normal          normal             0.000139            0.999861   

  response_ranger  prob.suspect_ranger  prob.normal_ranger  
0          normal             0.442594            0.557406  
1          normal             0.001667            0.998333  
2          normal             0.027838            0.972162  
3          normal             0.043218            0.956782  
4          normal             0.001630            0.998370  


In [65]:
data_eval_ncv_2 = pyreadr.read_r(os.path.join(os.getcwd(), "data_eval_ncv_2.rds"))[None]
print(data_eval_ncv_2.head())

   fold  row_ids   truth response_glmnet  prob.suspect_glmnet  \
0     1        1  normal          normal             0.000327   
1     1        3  normal          normal             0.040482   
2     1        4  normal          normal             0.063884   
3     1       23  normal          normal             0.083980   
4     1       35  normal          normal             0.009852   

   prob.normal_glmnet response_ranger  prob.suspect_ranger  prob.normal_ranger  
0            0.999673          normal             0.000725            0.999275  
1            0.959518          normal             0.466339            0.533661  
2            0.936116          normal             0.177751            0.822249  
3            0.916020          normal             0.087877            0.912123  
4            0.990148          normal             0.011840            0.988160  


In [67]:
data_eval_tmp_2 = pyreadr.read_r(os.path.join(os.getcwd(), "data_eval_tmp_2.rds"))[None]
data_eval_tmp_2.head()

,fold,row_ids,truth,response_glmnet,prob.suspect_glmnet,prob.normal_glmnet,response_ranger,prob.suspect_ranger,prob.normal_ranger
0,1,314,normal,suspect,0.974549,0.025451,normal,0.098596,0.901404
1,1,315,normal,suspect,0.958650,0.041350,normal,0.087827,0.912173
2,1,316,normal,suspect,0.999728,0.000272,normal,0.063504,0.936496
3,1,317,normal,normal,0.000050,0.999950,normal,0.001771,0.998229
4,1,318,normal,suspect,0.999372,0.000628,normal,0.050645,0.949355


This leaves us with 3 evaluation scenarios:

- ttt: models tuned via simple hold-out, the two respective winners (glmnet vs. ranger) are evaluated on the test data (after refitting)
- ncv: models tuned in inner loop of nested CV, the two respective winners (glmnet vs. ranger) are evaluated in the outer loop (after refitting, one model for each of the 5 folds)
- tmp: for a sensitivity analysis, we use the respective best hyperparameters for the glmnet and random forest from the train-tune-test design and evaluate the "winners" again for the temporal split (train one year, evaluate next year)
(note that for a really consistent evaluation an independent "inner" loop is missing here...)

## 2.6 (Hypothetical) comparison of evaluation designs

For a real-world ML problem, we would decide for a single experimental design a priori. 
In this exercise, we will compare different evaluation approaches for learning purposes. 

### Task

For each design mentioned in section 2.5, calculate test performance estimates, initially for the AUC. 
(the function `roc_auc_score` that we already used previously should be of some help here)
Here, you should report the individual performances (ranger, glmnet) and, in addition, the difference
of performances for comparison (ranger - glmnet).

Note that for nested CV, you have to write a small custom loop or function that calculate the metric
per fold and then average the results for an overall (expected) performance assessment.

When you are done, repeat with other metrics (tasks, design, ...) and try to interpret the results.

In [86]:
# You can use this codebox for your solution or add additional ones if needed

def evaluate(data_eval):
    random_forest_auc = roc_auc_score(data_eval['truth'], data_eval['prob.suspect_ranger'], labels = ["normal", "suspect"])
    glmnet_auc = roc_auc_score(data_eval['truth'], data_eval['prob.suspect_glmnet'], labels = ["normal", "suspect"])
    diff_auc = random_forest_auc - glmnet_auc
    return pd.DataFrame({
        "auc_ranger": [random_forest_auc],
        "auc_glmnet": [glmnet_auc],
        "auc_delta": [diff_auc]
    })

def evaluate_cv(data_eval, aggr = pd.DataFrame.mean):
    data_eval_grouped = data_eval.groupby("fold").apply(evaluate, include_groups=False)
    return pd.DataFrame(aggr(data_eval_grouped, axis = 0)).T

results_ignoring_folds = evaluate(data_eval_ttt_2).assign(name = "ttt")
results_ignoring_folds = pd.concat([results_ignoring_folds, evaluate(data_eval_ncv_2).assign(name = "ncv"), evaluate(data_eval_tmp_2).assign(name = "tmp")], ignore_index=True)
results_ignoring_folds = results_ignoring_folds.set_index("name")
results_ignoring_folds


,auc_ranger,auc_glmnet,auc_delta
name,,,
ttt,0.985714,0.980711,0.005003
ncv,0.990150,0.963510,0.026641
tmp,0.957494,0.930004,0.027490


In [87]:
valid_results = evaluate(data_eval_ttt_2).assign(name = "ttt")
valid_results = pd.concat([valid_results, evaluate_cv(data_eval_ncv_2).assign(name = "ncv")], ignore_index=True)
valid_results = pd.concat([valid_results, evaluate_cv(data_eval_tmp_2).assign(name = "tmp")], ignore_index=True)
valid_results = valid_results.set_index("name")

In [88]:
valid_results

,auc_ranger,auc_glmnet,auc_delta
name,,,
ttt,0.985714,0.980711,0.005003
ncv,0.990057,0.963949,0.026108
tmp,0.945188,0.910174,0.035013
